In [9]:
import os
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import math
print(torch.__version__)
plt.ion()   # interactive mode
from torch.autograd import Variable
import cv2
from scipy import stats

1.4.0


In [2]:
transform_ori = transforms.Compose([transforms.RandomHorizontalFlip(),    #flipping the image horizontally
                                    transforms.RandomResizedCrop(64),
                                    transforms.ToTensor(), 
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
'''
rootdir = './test/no/'
for file in os.listdir(rootdir):
    im = Image.open(rootdir+file)
    img = transform_ori(im)
    inputs = Variable(img)
#     print (img)
#     print(len(img))
#     print(len(img[0]))
#     print(len(img[0][0]))
    break
'''

"\nrootdir = './test/no/'\nfor file in os.listdir(rootdir):\n    im = Image.open(rootdir+file)\n    img = transform_ori(im)\n    inputs = Variable(img)\n#     print (img)\n#     print(len(img))\n#     print(len(img[0]))\n#     print(len(img[0][0]))\n    break\n"

In [3]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        # sub sampling - extract features
        self.cnn1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3,stride=1, padding=1)
        self.batchnorm1 = nn.BatchNorm2d(8)        #Batch normalization
        self.relu = nn.ReLU()                 #RELU Activation
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)   #Maxpooling reduces the size by kernel size. 64/2 = 32
        
        self.cnn2 = nn.Conv2d(in_channels=8, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.batchnorm2 = nn.BatchNorm2d(32)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)    #Size now is 32/2 = 16
        
        self.cnn3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=2)
        self.batchnorm3 = nn.BatchNorm2d(64)
        self.maxpool3 = nn.MaxPool2d(kernel_size=2)    #Size now is 16/2 = 8
        
        # fully connected - classification
        #Flatten the feature maps. You have 32 feature mapsfrom cnn2. Each of the feature is of size 16x16 --> 32*16*16 = 8192
        self.fc1 = nn.Linear(in_features=4096, out_features=4000)   #Flattened image is fed into linear NN and reduced to half size
        self.droput = nn.Dropout(p=0.5)                    #Dropout used to reduce overfitting
        self.fc2 = nn.Linear(in_features=4000, out_features=2000)
        self.droput = nn.Dropout(p=0.5)
        self.fc3 = nn.Linear(in_features=2000, out_features=500)
        self.droput = nn.Dropout(p=0.5)
        self.fc4 = nn.Linear(in_features=500, out_features=50)
        self.droput = nn.Dropout(p=0.5)
        self.fc5 = nn.Linear(in_features=50, out_features=2)   
        self.softmax = nn.Softmax(dim = 1)
        
    def forward(self,x):
        out = self.cnn1(x)
        out = self.batchnorm1(out)
        out = self.relu(out)
        out = self.maxpool1(out)
        out = self.cnn2(out)
        out = self.batchnorm2(out)
        out = self.relu(out)
        out = self.maxpool2(out)
        out = self.cnn3(out)
        out = self.batchnorm3(out)
        out = self.relu(out)
        out = self.maxpool3(out)
        #Flattening is done here with .view() -> (batch_size, 32*16*16) = (100, 8192)
        out = out.view(-1,4096)   #-1 will automatically update the batchsize as 100; 8192 flattens 32,16,16
        #Then we forward through our fully connected layer 
        out = self.fc1(out)
        out = self.relu(out)
#         out = self.droput(out)
        out = self.fc2(out)
        out = self.relu(out)
#         out = self.droput(out)
        out = self.fc3(out)
        out = self.relu(out)
#         out = self.droput(out)
        out = self.fc4(out)
        out = self.relu(out)
#         out = self.droput(out)
        out = self.fc5(out)
        out = self.softmax(out)
        return out

In [4]:
# load pre-trained model
model = torch.load('./demo_images_95.pt')

In [5]:
'''
rootdir = './test/no/'

model.eval()                    # Put the network into evaluation mode
for file in os.listdir(rootdir):
    im = Image.open(rootdir+file)
    img = transform_ori(im)
    img_4d = img.unsqueeze(0)
    inputs = Variable(img_4d)

    outputs = model(inputs)  

    _, predicted = torch.max(outputs, 1)
    print (predicted)
'''

"\nrootdir = './test/no/'\n\nmodel.eval()                    # Put the network into evaluation mode\nfor file in os.listdir(rootdir):\n    im = Image.open(rootdir+file)\n    img = transform_ori(im)\n    img_4d = img.unsqueeze(0)\n    inputs = Variable(img_4d)\n\n    outputs = model(inputs)  \n\n    _, predicted = torch.max(outputs, 1)\n    print (predicted)\n"

In [26]:
cap = cv2.VideoCapture('../../Downloads/short1outpy.avi')
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter('short1write.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))
font = cv2.FONT_HERSHEY_SIMPLEX 
minidict = {0:'NOTseeP',1:'seeP'}
i = 0
ans_list = []
result = 0
frame_interval = 10
while True:
# while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True: 
        left_right_image = np.split(frame, 2, axis=1)
        left_img = left_right_image[0]
        right_img = left_right_image[1]
        left_img_pil = Image.fromarray(left_img)
        right_img_pil = Image.fromarray(right_img)
    #     print (left_img_pil)
    #     print (type(left_img_pil))
#         cv2.imshow('frame',frame)
        left_img_pil2tensor = transform_ori(left_img_pil)
        right_img_pil2tensor = transform_ori(right_img_pil)
        left_img_4d = left_img_pil2tensor.unsqueeze(0)
        right_img_4d = right_img_pil2tensor.unsqueeze(0)
        left_inputs = Variable(left_img_4d)
        right_inputs = Variable(right_img_4d)
        left_outputs = model(left_inputs) 
        right_outputs = model(right_inputs)  
        _, left_predicted = torch.max(left_outputs, 1)
        _, right_predicted = torch.max(right_outputs, 1)
#         print ([left_predicted,right_predicted])
        left_ans = left_predicted.numpy()[0]
        right_ans = right_predicted.numpy()[0]
        ans_list.append(left_ans)
        ans_list.append(right_ans)
#         cv2.putText(frame,str(left_ans), (50, 50), font, 1, (0, 255, 255), 2, cv2.LINE_4) # left 
#         cv2.putText(frame,str(right_ans), (1500, 50), font, 1, (0, 255, 255), 2, cv2.LINE_4) # right

        if i % frame_interval == frame_interval-1 and i >= frame_interval-1:
            result = stats.mode(ans_list)[0][0]
#             print (str(result))
            cv2.putText(frame,minidict[result], (50, 50), font, 1, (0, 255, 255), 2, cv2.LINE_4)
            ans_list = []
        elif i < frame_interval-1:
            if left_ans == right_ans:
                cv2.putText(frame,minidict[left_ans], (50, 50), font, 1, (0, 255, 255), 2, cv2.LINE_4)
            else:
                cv2.putText(frame,minidict[0], (50, 50), font, 1, (0, 255, 255), 2, cv2.LINE_4)
        else:
            cv2.putText(frame,minidict[result], (50, 50), font, 1, (0, 255, 255), 2, cv2.LINE_4)
        cv2.putText(frame,str(i), (1230, 50), font, 1, (0, 255, 255), 2, cv2.LINE_4)
        out.write(frame)
        i += 1
    #     break
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     break
    else:
        break
cap.release()
out.release()
cv2.destroyAllWindows()

In [37]:
print (len(left_img))
print (len(left_img[0]))
print (len(left_img[0][0]))

376
672
3
